<h1>1. Import Libraries</h1>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import time
import numpy as np
import os
import cv2

<h1>2. Data Preprocessing</h1>

In [2]:
# Function to load and preprocess the dataset
def load_data(csv_file, img_dir, transform=None):
    data = pd.read_csv(csv_file)
    images, physical_features, labels = [], [], []

    for idx in range(len(data)):
        img_id = str(data.iloc[idx]['id'])
        img_path = os.path.join(img_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        
        if transform:
            image = transform(image)
        
        stable_height = int(data.iloc[idx]['stable_height']) - 1  # zero-based class index
        block_type = data.iloc[idx]['type']
        cam_angle = data.iloc[idx]['cam_angle']

        images.append(image)
        physical_features.append([block_type, cam_angle])
        labels.append(stable_height)
    
    return images, torch.tensor(physical_features, dtype=torch.float32), torch.tensor(labels)


<h1>3. Model Definition</h1>

In [3]:
import torch
import torch.nn as nn
from torchvision import models

# Define the model
class CustomResNetWithFeatures(nn.Module):
    def __init__(self, num_types, num_classes):
        super(CustomResNetWithFeatures, self).__init__()
        self.resnet = models.resnet18(pretrained=True)  # Using ResNet18
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 128)  # Modify ResNet output to 128 features

        # Embedding for block type
        self.type_embedding = nn.Embedding(num_types, 10)  # Embedding for block types
        
        # Fully connected layers for classification
        self.fc1 = nn.Linear(128 + 10 + 1, 64)  # Combine image features, block type embedding, and cam_angle
        self.fc2 = nn.Linear(64, num_classes)  # Final output for classification (logits for each class)

    def forward(self, x, block_type, cam_angle):
        x = self.resnet(x)  # Pass through ResNet
        type_embed = self.type_embedding(block_type)  # Get block type embeddings
        cam_angle = cam_angle.unsqueeze(1)  # Ensure cam_angle has correct shape
        
        # Concatenate features
        combined = torch.cat((x, type_embed, cam_angle), dim=1)
        
        x = torch.relu(self.fc1(combined))  # Pass through the first fully connected layer
        logits = self.fc2(x)  # Get final class logits
        return logits

# Update get_model function to return the model
def get_model(num_types, num_classes):
    return CustomResNetWithFeatures(num_types, num_classes)


<h1>4. Training Function</h1>

In [4]:
def train_model(csv_file_path, images_dir_path, num_types, num_classes=6, num_epochs=10, batch_size=16, model_save_path="model.pth"):
    # Define image transformations
    transform = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load data
    images, physical_features, labels = load_data(csv_file_path, images_dir_path, transform)
    dataset = list(zip(images, physical_features, labels))
    
    # Use DataLoader without the need to stack tensors manually
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize model and optimizer
    model = get_model(num_types, num_classes)
    criterion = nn.CrossEntropyLoss()  # For classification
    optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-4)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_correct = 0
        model.train()

        for batch in dataloader:
            images, physical_features, stable_heights = batch
            images = images.to(device)  # No need to stack, already a tensor
            block_type = (physical_features[:, 0] - 1).long().to(device)
            cam_angle = physical_features[:, 1].to(device)
            stable_heights = stable_heights.to(device)

            optimizer.zero_grad()
            logits = model(images, block_type, cam_angle)
            loss = criterion(logits, stable_heights)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted_classes = torch.max(logits, 1)
            running_correct += (predicted_classes == stable_heights).sum().item()

        accuracy = running_correct / len(dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader.dataset):.4f}, Accuracy: {accuracy:.4f}")

        # Save the model after every epoch
        torch.save(model.state_dict(), f"{model_save_path}_{epoch+1}.pth")
        print(f"Model saved after Epoch {epoch+1} to {model_save_path}")
    
    return model

<h1>5. Saving and Loading Model</h1>

In [5]:
# Function to save the model
def save_model(model, model_path):
    torch.save({'model_state_dict': model.state_dict()}, model_path)

# Function to load the model
def load_model(model_path, num_types):
    model = get_model(num_types, num_classes=6)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()  # Set to evaluation mode
    return model


<h1>7. Running the Training</h1>

In [6]:
def load_model(model_save_path, num_types, num_classes):
    # Initialize the model
    model = get_model(num_types, num_classes)
    
    # Load the saved model state dict
    model.load_state_dict(torch.load(model_save_path))
    model.eval()  # Set the model to evaluation mode (important for inference)
    
    return model


In [7]:
csv_file_path = '../COMP90086_2024_Project_train/train.csv'  # Path to your training CSV
images_dir_path = '../COMP90086_2024_Project_train/train'  # Path to your training images
model = train_model(csv_file_path, images_dir_path, num_types=2, num_classes=6, model_save_path='models/model')


/home/theo/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/theo/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.0926, Accuracy: 0.3480
Model saved after Epoch 1 to models/model
Epoch [2/10], Loss: 0.0771, Accuracy: 0.4878
Model saved after Epoch 2 to models/model
Epoch [3/10], Loss: 0.0675, Accuracy: 0.5650
Model saved after Epoch 3 to models/model
Epoch [4/10], Loss: 0.0589, Accuracy: 0.6376
Model saved after Epoch 4 to models/model
Epoch [5/10], Loss: 0.0482, Accuracy: 0.7094
Model saved after Epoch 5 to models/model
Epoch [6/10], Loss: 0.0371, Accuracy: 0.7823
Model saved after Epoch 6 to models/model
Epoch [7/10], Loss: 0.0275, Accuracy: 0.8417
Model saved after Epoch 7 to models/model
Epoch [8/10], Loss: 0.0203, Accuracy: 0.8819
Model saved after Epoch 8 to models/model
Epoch [9/10], Loss: 0.0163, Accuracy: 0.9072
Model saved after Epoch 9 to models/model
Epoch [10/10], Loss: 0.0117, Accuracy: 0.9380
Model saved after Epoch 10 to models/model


In [8]:
model_ = load_model('model.pth', num_types=2, num_classes=6)

/tmp/ipykernel_4837/1111035268.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


In [9]:
def load_images_from_folder(folder_path, ids, image_size=(224, 224)):
    images = []
    for id in ids:
        filename = str(int(id)) + '.jpg'

        # Load the image using OpenCV
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)

        # Resize the image to the target size (224, 224)
        img = cv2.resize(img, image_size)

        # Convert BGR (OpenCV default) to RGB if needed (TensorFlow models use RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Normalize pixel values to the range [0, 1]
        img_array = img / 255.0

        images.append(img_array)

    # Convert the list of images to a NumPy array with shape (n, 224, 224, 3)
    return np.array(images)

folder_path = '../COMP90086_2024_Project_test/test'
test = np.genfromtxt('../COMP90086_2024_Project_test/test.csv', delimiter=',', skip_header=1)
ids = np.array(test, dtype=int)
test_image_data = load_images_from_folder(folder_path, ids)

# Predict stable height on the test set

In [10]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

# 2. Preprocess the image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match training size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Apply the same normalization used during training
    ])
    
    # Load image using PIL
    image = Image.open(image_path).convert('RGB')
    
    # Apply transformations and return a batch with a single image (unsqueeze to add batch dimension)
    return transform(image).unsqueeze(0)

# 3. Make a prediction
def predict_image(model, image_tensor, block_type, cam_angle):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    image_tensor = image_tensor.to(device)
    block_type = torch.tensor([block_type]).to(device).long()  # Pass as tensor
    cam_angle = torch.tensor([cam_angle]).to(device).float()   # Pass as tensor
    
    with torch.no_grad():  # Disable gradient calculations for inference
        logits = model(image_tensor, block_type, cam_angle)
        predicted_class = torch.argmax(logits, dim=1).item() + 1  # Convert prediction to class (add 1 for class range [1-6])
    
    return predicted_class

# 4. Example usage
model_path = 'model.pth'  # Path to the saved model
folder_path = '../COMP90086_2024_Project_test/test'  # Path to the input image
num_types = 2  # Adjust based on your dataset
num_classes = 6  # Number of stable_height classes

# Load model
model = load_model(model_path, num_types, num_classes)

predictions = {}
predictions["id"] = []
predictions["stable_height"] = []

for image in os.listdir(folder_path):
    image_id = image.strip(".jpg")
    image_tensor = preprocess_image(f"{folder_path}/{image}")


    # Predict stable height (assuming default block_type=1, cam_angle=0.0, adjust as needed)
    predicted_class = predict_image(model, image_tensor, block_type=1, cam_angle=0.0)

    print(f"{image_id}: {predicted_class}")
    predictions["id"].append(image_id)
    predictions["stable_height"].append(predicted_class)


/tmp/ipykernel_4837/1111035268.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


100156: 1
100357: 6
100998: 1
102119: 2
102131: 5
102213: 3
10265: 2
103351: 3
103671: 4
104154: 6
104298: 6
104408: 5
105572: 2
105753: 1
106117: 2
106155: 4
106207: 2
106615: 4
106839: 1
107131: 1
107154: 1
108129: 6
108162: 3
108306: 1
108313: 6
109548: 6
110737: 1
111486: 3
111571: 5
111936: 3
112148: 2
112408: 3
11329: 2
114074: 2
114456: 3
11485: 2
115147: 5
116446: 2
116758: 4
117059: 6
117554: 3
117913: 3
118233: 2
118392: 1
118644: 5
119874: 3
120797: 2
121757: 2
123349: 3
123544: 4
123994: 6
125981: 4
126835: 6
127129: 3
128888: 6
129188: 4
129287: 1
129464: 4
129853: 3
129905: 4
130200: 3
130227: 3
130325: 4
130451: 3
130956: 5
131307: 6
131690: 6
131853: 1
132664: 2
132993: 2
133736: 2
133791: 2
133859: 1
134412: 3
135144: 2
137131: 2
138612: 5
138697: 6
13901: 3
139770: 5
140013: 4
140787: 1
142146: 2
143614: 1
143947: 2
144874: 3
145326: 3
146340: 2
146432: 6
146463: 1
147577: 6
147836: 4
148326: 5
148338: 5
148342: 1
148680: 4
149231: 3
149364: 1
149931: 3
150322: 4
1514

In [11]:
df = pd.DataFrame(predictions)
df.to_csv("pred.csv", index=False)